In [1]:
print("Importing numpy.")
import numpy as np

print("Importing matplotlib.")
import matplotlib
matplotlib.rc('xtick', labelsize=14) 
matplotlib.rc('ytick', labelsize=14) 
from matplotlib import gridspec
import matplotlib.pyplot as plt
%matplotlib widget
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.sans-serif'] = ['Times New Roman']

print("Importing ipy.")
from ipywidgets import interact, widgets, FloatSlider, Dropdown, Checkbox, VBox
from IPython.display import display, clear_output

print("Importing PIL.")
from PIL import Image

print("Importing pandas.")
from pandas import DataFrame

import warnings
warnings.filterwarnings("ignore")

print("Importing keras.")
from keras.models import model_from_json

Importing numpy.
Importing matplotlib.
Importing ipy.
Importing PIL.
Importing pandas.
Importing keras.


In [32]:
## LOAD MODEL
model_dir = f'models/classifier/reco/model/'
json_file = open(model_dir + f'model_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights(model_dir + f'model_1.h5')

## LOAD INPUTS
file = np.load("inputs/Reco_Inputs/nn_input_MX700_MY400_class.npz")
nevents = len(np.concatenate((file['test'], file['train'], file['val'])))

In [45]:
x_test = file['x_test']
X_test = file['X_test']
ntest = len(file['test'])

scores = model.predict(x_test)
scores = scores[:,0]
rscores = np.around(scores, decimals=3)

## APPLY MODEL TO INPUTS
pairings = []
for i in range(15):
    pairings.append(scores[np.arange(i, ntest, 15)])

events = []
for i in range(0, ntest*15, 15):
    events.append(scores[i:i+15])

In [46]:
H_pairs = [0, 1, 2]

def get_ratio(pair, threshold):
    return np.sum(scores[pair] > cut)/len(scores[pair])

empty =  []
d = {i:empty for i in range(15)}
d['3H'] = []

thresholds = np.arange(0.0,1.01,0.01)
for cut in thresholds:
    for i in range(15):
        pair = np.arange(i,len(scores),15)
        eff = get_ratio(pair, cut)
        d[i] = d[i] + [eff]

    X_mask = np.arange(0,len(scores),15)
    Y1_mask = np.arange(1,len(scores),15)
    Y2_mask = np.arange(2,len(scores),15)
    mask = X_mask & Y1_mask & Y2_mask
    d['3H'] = d['3H'] +  [get_ratio(mask, cut)]


In [47]:
indices = np.array((), int)
for ind in np.argwhere(rscores == 0):
    indices = np.append(indices, ind)

In [48]:
label = [r'$H_X$', r'$H_{Y,1}$', r'$H_{Y,2}$', r'$b_{X,1}$, $b_{Y_1,1}$', r'$b_{X,1}$, $b_{Y_1,2}$', r'$b_{X,1}$, $b_{Y_2,1}$', r'$b_{X,1}$, $b_{Y_2,2}$', r'$b_{X,2}$, $b_{Y_1,1}$', r'$b_{X,2}$, $b_{Y_1,2}$', r'$b_{X,2}$, $b_{Y_2,1}$', r'$b_{X,2}$, $b_{Y_2,2}$', r'$b_{Y_1,1}$, $b_{Y_2,1}$', r'$b_{Y_1,1}$, $b_{Y_2,2}$', r'$b_{Y_1,2}$, $b_{Y_2,1}$', r'$b_{Y_1,2}$, $b_{Y_2,2}$']
di = {}
di[r'$3H$'] = d['3H']
for i in np.arange(0,15):
    di[label[i]] = d[i]

thresholds = np.round(thresholds, 2)
df = DataFrame.from_dict(data=di, orient='index', columns=thresholds).round(decimals=3)
df

,0.00,0.01,0.02,0.03,0.04,0.05,0.06,0.07,0.08,0.09,...,0.91,0.92,0.93,0.94,0.95,0.96,0.97,0.98,0.99,1.00
$3H$,1.0,0.786,0.757,0.740,0.727,0.715,0.707,0.699,0.692,0.685,...,0.322,0.314,0.304,0.291,0.277,0.257,0.231,0.192,0.086,0.0
$H_X$,1.0,0.987,0.985,0.983,0.980,0.978,0.977,0.975,0.974,0.973,...,0.731,0.717,0.696,0.673,0.647,0.611,0.558,0.466,0.220,0.0
"$H_{Y,1}$",1.0,0.992,0.990,0.989,0.988,0.988,0.987,0.987,0.986,0.986,...,0.825,0.811,0.794,0.772,0.746,0.711,0.664,0.568,0.281,0.0
"$H_{Y,2}$",1.0,0.968,0.957,0.951,0.944,0.940,0.936,0.931,0.927,0.922,...,0.304,0.281,0.255,0.235,0.213,0.191,0.163,0.116,0.032,0.0
"$b_{X,1}$, $b_{Y_1,1}$",1.0,0.158,0.137,0.125,0.119,0.114,0.108,0.105,0.102,0.098,...,0.027,0.025,0.024,0.022,0.019,0.017,0.014,0.011,0.006,0.0
"$b_{X,1}$, $b_{Y_1,2}$",1.0,0.465,0.432,0.407,0.394,0.384,0.373,0.364,0.356,0.349,...,0.094,0.089,0.082,0.076,0.069,0.061,0.052,0.042,0.017,0.0
"$b_{X,1}$, $b_{Y_2,1}$",1.0,0.573,0.542,0.521,0.507,0.495,0.487,0.479,0.470,0.463,...,0.176,0.169,0.162,0.151,0.141,0.127,0.110,0.086,0.029,0.0
"$b_{X,1}$, $b_{Y_2,2}$",1.0,0.742,0.709,0.688,0.673,0.660,0.649,0.640,0.629,0.623,...,0.214,0.202,0.190,0.177,0.167,0.149,0.129,0.104,0.054,0.0
"$b_{X,2}$, $b_{Y_1,1}$",1.0,0.198,0.166,0.151,0.139,0.130,0.121,0.118,0.113,0.109,...,0.011,0.011,0.010,0.009,0.008,0.007,0.006,0.004,0.001,0.0
"$b_{X,2}$, $b_{Y_1,2}$",1.0,0.621,0.579,0.551,0.529,0.512,0.497,0.484,0.471,0.462,...,0.050,0.046,0.041,0.037,0.033,0.030,0.024,0.016,0.003,0.0


In [49]:
def get_bars(val):
    return [d[i][val] for i in np.arange(15)]

def update(val = 1):
    y = get_bars(val)
    
    ax.bar(x, y)
    fig.canvas.draw_idle()

In [50]:
x_dict = {'HX':0,
 'HY1':1,
 'HY2':2,
 'X b1, Y1 b1':3,
 'X b1, Y1 b2':4,
 'X b1, Y2 b1':5,
 'X b1, Y2 b2':6,
 'X b2, Y1 b1':7,
 'X b2, Y1 b2':8,
 'X b2, Y2 b1':9,
 'X b2, Y2 b2':10,
 'Y1 b1, Y2 b1':11,
 'Y1 b1, Y2 b2':12,
 'Y1 b2, Y2 b1':13,
 'Y1 b2, Y2 b2':14}

In [51]:
x_drop = ['HX',
 'HY1',
 'HY2',
 'X b1, Y1 b1',
 'X b1, Y1 b2',
 'X b1, Y2 b1',
 'X b1, Y2 b2',
 'X b2, Y1 b1',
 'X b2, Y1 b2',
 'X b2, Y2 b1',
 'X b2, Y2 b2',
 'Y1 b1, Y2 b1',
 'Y1 b1, Y2 b2',
 'Y1 b2, Y2 b1',
 'Y1 b2, Y2 b2']

In [52]:
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection

In [53]:
def dropdown_event(change):
    global a
    a.remove()
    pos = x_dict[change.new] + 1
    arrow = mpatches.Arrow(pos, 1.2, 0, -0.2, color='blue')
    a = bar_ax.add_patch(arrow)
    
    X_bbar .set_visible(False)
    X_b.set_visible(False)
    Y1_bbar.set_visible(False)
    Y1_b.set_visible(False)
    Y2_bbar.set_visible(False)
    Y2_b.set_visible(False)
    
    if 'b' in change.new:
        part1, part2 = change.new.split(',')
        if ('X' in part1 and 'b1' in part1) or ('X' in part2 and 'b1' in part2):
            X_b.set_visible(True)
        if ('X' in part1 and 'b2' in part1) or ('X' in part2 and 'b2' in part2):
            X_bbar.set_visible(True)
        if ('Y1' in part1 and 'b1' in part1) or ('Y1' in part2 and 'b1' in part2):
            Y1_b.set_visible(True)
        if ('Y1' in part1 and 'b2' in part1) or ('Y1' in part2 and 'b2' in part2):
            Y1_bbar.set_visible(True)
        if ('Y2' in part1 and 'b1' in part1) or ('Y2' in part2 and 'b1' in part2):
            Y2_b.set_visible(True)
        if ('Y2' in part1 and 'b2' in part1) or ('Y2' in part2 and 'b2' in part2):
            Y2_bbar.set_visible(True)
    elif 'H' in change.new:
        if 'X' in change.new:
            X_b.set_visible(True)
            X_bbar.set_visible(True)
        if 'Y1' in change.new:
            Y1_b.set_visible(True)
            Y1_bbar.set_visible(True)
        if 'Y2' in change.new:
            Y2_b.set_visible(True)
            Y2_bbar.set_visible(True)
            
def f(threshold):
    global fig, bar_ax
    bar_ax.clear()
    bar_ax.set_ylim(0,1.1)
    bar_ax.set_xlim(-1,16)
    bar_ax.set_ylabel('Ratio of events above threshold', fontsize=18)
    bar_ax.set_xlabel('Pairs', fontsize=18)
    bar_ax.set_xticklabels(labels=[r'$3H$'] + label, rotation=45,  ha='right')
    y = get_bars(int(threshold*100))
    Higgs = [d['3H'][int(threshold*100)]] + [y[0], y[1], y[2]] + [0]*12
    Non_Higgs = [0]*4 + y[3:]
    Higgs_bars = bar_ax.bar(x, Higgs, color='C2')
    nonHiggs_bars = bar_ax.bar(x, Non_Higgs, color='C3')
    fig.canvas.draw() 
    renderer = fig.canvas.renderer 
    bar_ax.draw(renderer)
    plt.tight_layout()
    plt.show()

In [54]:
threshold = 0.5
fig = plt.figure(figsize=(12,6))
gs = gridspec.GridSpec(1, 2, width_ratios=[6, 5]) 
bar_ax = fig.add_subplot(gs[0])
x = [r'$3H$'] + label
y = get_bars(int(threshold*100))
Higgs = [d['3H'][int(threshold*100)]] + [y[0], y[1], y[2]] + [0]*12
Non_Higgs = [0]*4 + y[3:]
Higgs_bars = bar_ax.bar(x, Higgs, color='C2')
nonHiggs_bars = bar_ax.bar(x, Non_Higgs, color='C3')
bar_ax.set_ylim(0,1.1)
bar_ax.set_xlim(-1,16)
bar_ax.set_ylabel('Ratio of events above threshold', fontsize=18)
bar_ax.set_xlabel('Pairs', fontsize=18)
bar_ax.set_xticklabels(labels=[r'$3H$'] + label, rotation=45,  ha='right')

arrow = mpatches.Arrow(1, 1.2, 0, -0.2, color='blue')
a = bar_ax.add_patch(arrow)

ax = fig.add_subplot(gs[1], frameon=False)
ax.axis('off')
ax.set_ylim(0,1)
ax.set_xlim(0,1)

X_bbar  = mpatches.Circle((0.770, 0.150), 0.04, color='blue', fill=False, zorder=10)
X_b     = mpatches.Circle((0.770, 0.320), 0.04, color='blue', fill=False, zorder=10)
Y1_bbar = mpatches.Circle((0.945, 0.375), 0.04, color='blue', fill=False, zorder=10)
Y1_b    = mpatches.Circle((0.945, 0.545), 0.04, color='blue', fill=False, zorder=10)
Y2_bbar = mpatches.Circle((0.945, 0.675), 0.04, color='blue', fill=False, zorder=10)
Y2_b    = mpatches.Circle((0.945, 0.845), 0.04, color='blue', fill=False, zorder=10)

X_bbar.set_visible(True)
X_b.set_visible(True)
Y1_bbar.set_visible(False)
Y1_b.set_visible(False)
Y2_bbar.set_visible(False)
Y2_b.set_visible(False)

ax.add_patch(X_bbar)
ax.add_patch(X_b)
ax.add_patch(Y1_bbar)
ax.add_patch(Y1_b)
ax.add_patch(Y2_bbar)
ax.add_patch(Y2_b)

image = Image.open('CascadeDecay.png')
im = OffsetImage(image, zoom=0.35)
ab = AnnotationBbox(im, (0.5,0.5), xycoords='data', boxcoords="offset points",  pad=0.3,  arrowprops=dict(arrowstyle="->"))
ax.add_artist(ab)

pair = Dropdown(options = x_drop)
    
interact(f, threshold=FloatSlider(value=0.5, min=0, max=1.0, step=0.01, continuous_update=False))
pair.observe(dropdown_event, names='value')
display(pair)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='threshold', max=1.0, step=0…

Dropdown(options=('HX', 'HY1', 'HY2', 'X b1, Y1 b1', 'X b1, Y1 b2', 'X b1, Y2 b1', 'X b1, Y2 b2', 'X b2, Y1 b1…

In [55]:
from consistent_plots import hist

In [57]:
threshold = 0.5
bins = np.linspace(0,5,100)
fig, ax = plt.subplots(figsize=(8,5))
ax.set_xlim(0,5)
ax.set_ylim(0,400)
showing = np.array(([True]*15), dtype=bool)
colors = [f'C{i}' for i in range(15)]

for i in range(15):
    pair_mask = np.arange(i,ntest*15,15)
    data = X_test[pair_mask]
    sc = scores[pair_mask]
    # 6 for dR
    n, bins, patches = hist(ax, data[sc > threshold,6], label=x_drop[i], color=colors[i], bins=bins)
ax.legend()

def f(cut, c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14):
    global fig, ax, threshold, bins
    showing = [c0, c1, c2, c3, c4, c5, c6, c7, c8, c9, c10, c11, c12, c13, c14]
    threshold = cut
    ax.clear()
    ax.set_xlim(0,5)
    ax.set_ylim(0,400)
    for i in range(15):
        if showing[i] == True:
            pair_mask = np.arange(i,ntest*15,15)
            data = X_test[pair_mask]
            sc = scores[pair_mask]
            # 6 for dR
            n, bins, patches = hist(ax, data[sc > threshold,6], label=x_drop[i], color=colors[i], bins=bins)
    ax.legend(np.array((x_drop))[showing])
    fig.canvas.draw() 
    renderer = fig.canvas.renderer 
    ax.draw(renderer)
    plt.tight_layout()
    plt.show()
    
slider = FloatSlider(value=0.5, min=0, max=1.0, step=0.01, continuous_update=False)
checkboxes = [widgets.ToggleButton(value=True, description=label) for label in x_drop]

i = widgets.interactive(f, 
                        cut=slider, 
                        c0=checkboxes[0],   c1=checkboxes[1],   c2=checkboxes[2], 
                        c3=checkboxes[3],   c4=checkboxes[4],   c5=checkboxes[5],
                        c6=checkboxes[6],   c7=checkboxes[7],   c8=checkboxes[8],
                        c9=checkboxes[9],   c10=checkboxes[10], c11=checkboxes[11],
                        c12=checkboxes[12], c13=checkboxes[13], c14=checkboxes[14])

hbox1 = widgets.HBox(checkboxes[:4])
hbox2 = widgets.HBox(checkboxes[4:8])
hbox3 = widgets.HBox(checkboxes[8:12])
hbox4 = widgets.HBox(checkboxes[12:])
vbox = VBox([slider, hbox1, hbox2, hbox3, hbox4])
vbox

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …